<a href="https://colab.research.google.com/github/bhanuteja2001/SegPC-2021/blob/main/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [28]:
import os
for file in (os.listdir("/content/drive/MyDrive")[3:]):
  if os.path.exists(str(file)):
    os.remove(str(file))
  else:
    print("The file {} does not exist",file)

In [29]:
os.listdir("/content/drive/MyDrive")

['covid_image_data', 'Colab Notebooks', 'SegPC-2021']

In [31]:
len(os.listdir("/content/drive/MyDrive/SegPC-2021/train/x"))

297

In [35]:
import os
import random
import re
from PIL import Image

DATA_PATH = '/content/drive/MyDrive/SegPC-2021/train/'
FRAME_PATH = "/content/drive/MyDrive/SegPC-2021/train/x/"
MASK_PATH = "/content/drive/MyDrive/SegPC-2021/train/y/"



# Create folders to hold images and masks

folders = ['train_frames', 'train_masks', 'val_frames', 'val_masks', 'test_frames', 'test_masks']


for folder in folders:
  os.makedirs(DATA_PATH + folder)
  


# Get all frames and masks, sort them, shuffle them to generate data sets.

all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)


all_frames.sort(key=lambda var:[int(x) if x.isdigit() else x 
                                for x in re.findall(r'[^0-9]|[0-9]+', var)])
all_masks.sort(key=lambda var:[int(x) if x.isdigit() else x 
                               for x in re.findall(r'[^0-9]|[0-9]+', var)])


random.seed(230)
random.shuffle(all_frames)


# Generate train, val, and test sets for frames

train_split = int(0.7*len(all_frames))
val_split = int(0.9 * len(all_frames))

train_frames = all_frames[:train_split]
val_frames = all_frames[train_split:val_split]
test_frames = all_frames[val_split:]


# Generate corresponding mask lists for masks

train_masks = [f for f in all_masks if f in train_frames]
val_masks = [f for f in all_masks if f in val_frames]
test_masks = [f for f in all_masks if f in test_frames]


#Add train, val, test frames and masks to relevant folders


def add_frames(dir_name, image):
  
  img = Image.open(FRAME_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'/'+image)
  
  
  
def add_masks(dir_name, image):
  
  img = Image.open(MASK_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'/'+image)


  
  
frame_folders = [(train_frames, 'train_frames'), (val_frames, 'val_frames'), 
                 (test_frames, 'test_frames')]

mask_folders = [(train_masks, 'train_masks'), (val_masks, 'val_masks'), 
                (test_masks, 'test_masks')]

# Add frames

for folder in frame_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)

  list(map(add_frames, name, array))
         
    
# Add masks

for folder in mask_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)
  

In [36]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        channel_shift_range=150.0,
        vertical_flip=True,
        horizontal_flip=True)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_image_generator = train_datagen.flow_from_directory(
'/content/drive/MyDrive/SegPC-2021/train/train_frames',
batch_size = #NORMALLY 4/8/16/32)

train_mask_generator = train_datagen.flow_from_directory(
'/content/drive/MyDrive/SegPC-2021/train/train_masks',
batch_size = #NORMALLY 4/8/16/32)

val_image_generator = val_datagen.flow_from_directory(
'/content/drive/MyDrive/SegPC-2021/train/val_frames',
batch_size = #NORMALLY 4/8/16/32)


val_mask_generator = val_datagen.flow_from_directory(
'/content/drive/MyDrive/SegPC-2021/train/val_masks',
batch_size = #NORMALLY 4/8/16/32)



train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)